In [3]:
import pymongo
import pandas as pd
import math
from datetime import datetime

# import dask.dataframe as dd ## 대규모 데이터 처리
# import vaex ## 대규모 데이터 처리

import os
from dotenv import load_dotenv

### 분석용데이터셋 생성
- youtube_user_id_inner 활용
- 날짜 맞춰서 테이블매핑 : 2023-03-26 ~ 2024-05-03

(1) 계정 분석 데이터셋
(2) 콘텐츠 분석 데이터셋

In [4]:
# 데이터 필터링 기준
# 날짜 기준
first_date = datetime(2023, 3, 26)
last_date = datetime(2024, 5, 3)

In [5]:
file_path = 'C:/py_src/awake/data/'

In [6]:
youtube_user_id_inner = pd.read_csv(file_path + 'youtube_user_id_inner.csv')
youtube_user_id_inner = list(youtube_user_id_inner['youtube_user_id'])

youtube_channel_demographics = pd.read_csv(file_path + 'youtube_channel_demographics.csv')

youtube_channel_locations = pd.read_csv(file_path + 'youtube_channel_locations.csv')
youtube_channel_locations = youtube_channel_locations.groupby(['youtube_user_id', 'end_date']).agg({'views': 'sum',
                                                                                                    'estimatedMinutesWatched': 'sum',
                                                                                                    'averageViewDuration': 'mean',
                                                                                                    'averageViewPercentage': 'mean'}).reset_index()

youtube_daily_channel_basics = pd.read_csv(file_path + 'youtube_daily_channel_basics.csv')
youtube_datas = pd.read_csv(file_path + 'youtube_datas.csv')
youtube_revenue = pd.read_csv(file_path + 'youtube_revenue.csv')
youtube_subscriber = pd.read_csv(file_path + 'youtube_subscriber.csv')
youtube_users = pd.read_csv(file_path + 'youtube_users.csv')
youtube_videos = pd.read_csv(file_path + 'youtube_videos.csv')

#### 테이블 현황 및 정리

In [7]:
# 계정 기준 필터링
youtube_revenue = youtube_revenue[youtube_revenue['youtube_user_id'].isin(youtube_user_id_inner)].sort_values(['youtube_user_id','data_created_at']).reset_index(drop=True)

# 날짜 기준 필터링
youtube_revenue['data_created_at'] = pd.to_datetime(youtube_revenue['data_created_at'])
opt_date = (youtube_revenue['data_created_at'] >= first_date) & (youtube_revenue['data_created_at'] <= last_date)
youtube_revenue = youtube_revenue[opt_date].sort_values(['youtube_user_id', 'data_created_at']).reset_index(drop=True)

print(youtube_revenue.columns)
print(youtube_revenue.shape)
print(youtube_revenue['data_created_at'].min(), youtube_revenue['data_created_at'].max())

# youtube_revenue.to_csv('C:/py_src/awake/data/youtube_revenue.csv',encoding='utf-8-sig',index=False)

Index(['youtube_user_id', 'data_created_at', 'estimated_revenue',
       'estimated_ad_revenue', 'estimated_red_partner_revenue',
       'gross_revenue', 'cpm', 'ad_impressions', 'monetized_playbacks',
       'playback_based_cpm'],
      dtype='object')
(85084, 10)
2023-03-26 00:00:00 2024-05-03 00:00:00


In [8]:
# 계정 기준 필터링
youtube_channel_demographics = youtube_channel_demographics[youtube_channel_demographics['youtube_user_id'].isin(youtube_user_id_inner)].sort_values(['youtube_user_id','end_date']).reset_index(drop=True)

# 날짜 기준 필터링
youtube_channel_demographics['end_date'] = pd.to_datetime(youtube_channel_demographics['end_date'])
opt_date = (youtube_channel_demographics['end_date'] >= first_date) & (youtube_channel_demographics['end_date'] <= last_date)
youtube_channel_demographics = youtube_channel_demographics[opt_date].sort_values(['youtube_user_id', 'end_date']).reset_index(drop=True)

print(youtube_channel_demographics.columns)
print(youtube_channel_demographics.shape)
print(youtube_channel_demographics['end_date'].min(), youtube_channel_demographics['end_date'].max())

# youtube_channel_demographics.to_csv('C:/py_src/awake/data/youtube_channel_demographics.csv',encoding='utf-8-sig',index=False)

Index(['youtube_user_id', 'end_date', 'age13-17.female', 'age13-17.male',
       'age18-24.female', 'age18-24.male', 'age25-34.female', 'age25-34.male',
       'age35-44.female', 'age35-44.male', 'age45-54.female', 'age45-54.male',
       'age55-64.female', 'age55-64.male', 'age65-.female', 'age65-.male'],
      dtype='object')
(70843, 16)
2023-03-26 00:00:00 2024-05-03 00:00:00


In [9]:
print(youtube_users.columns)
print(youtube_users.shape)

# youtube_users.to_csv('C:/py_src/awake/data/youtube_users.csv',encoding='utf-8-sig',index=False)

Index(['channel_id', 'channel_title', 'phone_num', 'report_user_id',
       'published_at', 'viewCount', 'subscriberCount', 'videoCount'],
      dtype='object')
(934, 8)


In [10]:
# 계정 기준 필터링
youtube_channel_locations = youtube_channel_locations[youtube_channel_locations['youtube_user_id'].isin(youtube_user_id_inner)].sort_values(['youtube_user_id','end_date']).reset_index(drop=True)

# 날짜 기준 필터링
youtube_channel_locations['end_date'] = pd.to_datetime(youtube_channel_locations['end_date'])
opt_date = (youtube_channel_locations['end_date'] >= first_date) & (youtube_channel_locations['end_date'] <= last_date)
youtube_channel_locations = youtube_channel_locations[opt_date].sort_values(['youtube_user_id', 'end_date']).reset_index(drop=True)

print(youtube_channel_locations.columns)
print(youtube_channel_locations.shape)
print(youtube_channel_locations['end_date'].min(), youtube_channel_locations['end_date'].max())

# youtube_channel_locations.to_csv('C:/py_src/awake/data/youtube_channel_locations.csv',encoding='utf-8-sig',index=False)

Index(['youtube_user_id', 'end_date', 'views', 'estimatedMinutesWatched',
       'averageViewDuration', 'averageViewPercentage'],
      dtype='object')
(80805, 6)
2023-03-26 00:00:00 2024-05-03 00:00:00


In [11]:
# 계정 기준 필터링
youtube_daily_channel_basics = youtube_daily_channel_basics[youtube_daily_channel_basics['youtube_user_id'].isin(youtube_user_id_inner)].sort_values(['youtube_user_id','day']).reset_index(drop=True)

# 날짜 기준 필터링
youtube_daily_channel_basics['day'] = pd.to_datetime(youtube_daily_channel_basics['day'])
opt_date = (youtube_daily_channel_basics['day'] >= first_date) & (youtube_daily_channel_basics['day'] <= last_date)
youtube_daily_channel_basics = youtube_daily_channel_basics[opt_date].sort_values(['youtube_user_id', 'day']).reset_index(drop=True)

print(youtube_daily_channel_basics.columns)
print(youtube_daily_channel_basics.shape)
print(youtube_daily_channel_basics['day'].min(), youtube_daily_channel_basics['day'].max())

# youtube_daily_channel_basics.to_csv('C:/py_src/awake/data/youtube_daily_channel_basics.csv',encoding='utf-8-sig',index=False)

Index(['youtube_user_id', 'day', 'annotationClickThroughRate',
       'annotationCloseRate', 'averageViewDuration', 'comments', 'dislikes',
       'estimatedMinutesWatched', 'likes', 'shares', 'subscribersGained',
       'subscribersLost', 'views', 'redViews', 'estimatedRevenue',
       'estimatedAdRevenue', 'estimatedRedPartnerRevenue', 'grossRevenue',
       'cpm', 'monetizedPlaybacks', 'adImpressions', 'playbackBasedCpm'],
      dtype='object')
(95717, 22)
2023-03-26 00:00:00 2024-05-03 00:00:00


In [12]:
# 계정 기준 필터링
youtube_subscriber = youtube_subscriber[youtube_subscriber['youtube_user_id'].isin(youtube_user_id_inner)].sort_values(['youtube_user_id','data_created_at']).reset_index(drop=True)

# 날짜 기준 필터링
youtube_subscriber['data_created_at'] = pd.to_datetime(youtube_subscriber['data_created_at'])
opt_date = (youtube_subscriber['data_created_at'] >= first_date) & (youtube_subscriber['data_created_at'] <= last_date)
youtube_subscriber = youtube_subscriber[opt_date].sort_values(['youtube_user_id', 'data_created_at']).reset_index(drop=True)

print(youtube_subscriber.columns)
print(youtube_subscriber.shape)
print(youtube_subscriber['data_created_at'].min(), youtube_subscriber['data_created_at'].max())

# youtube_subscriber.to_csv('C:/py_src/awake/data/youtube_subscriber.csv',encoding='utf-8-sig',index=False)

Index(['youtube_user_id', 'data_created_at', 'subscribers_gained',
       'subscribers_lost', 'subscribers_count'],
      dtype='object')
(95708, 5)
2023-03-26 00:00:00 2024-05-03 00:00:00


In [13]:
# 계정 기준 필터링
youtube_datas = youtube_datas[youtube_datas['youtube_user_id'].isin(youtube_user_id_inner)].sort_values(['youtube_user_id','data_created_at']).reset_index(drop=True)

# 날짜 기준 필터링
youtube_datas['data_created_at'] = pd.to_datetime(youtube_datas['data_created_at'])
opt_date = (youtube_datas['data_created_at'] >= first_date) & (youtube_datas['data_created_at'] <= last_date)
youtube_datas = youtube_datas[opt_date].sort_values(['youtube_user_id', 'data_created_at']).reset_index(drop=True)

print(youtube_datas.columns)
print(youtube_datas.shape)
print(youtube_datas['data_created_at'].min(), youtube_datas['data_created_at'].max())

# youtube_datas.to_csv('C:/py_src/awake/data/youtube_datas.csv',encoding='utf-8-sig',index=False)

Index(['youtube_user_id', 'data_created_at', 'published_at', 'channel_id',
       'channel_title', 'yt_search_keyword', 'subscriber_count', 'video_count',
       'view_count', 'comment_count', 'like_count', 'dislike_count',
       'estimated_minutes_watched', 'average_view_duration', 'status_code',
       'red_view_count', 'RELATED_VIDEO', 'SUBSCRIBER', 'YT_CHANNEL',
       'YT_SEARCH', 'PLAYLIST', 'YT_OTHER_PAGE', 'END_SCREEN', 'NO_LINK_OTHER',
       'EXT_URL', 'YT_PLAYLIST_PAGE', 'ANNOTATION', 'NOTIFICATION',
       'PRODUCT_PAGE', 'SOUND_PAGE', 'HASHTAGS', 'SHORTS', 'ADVERTISING',
       'UNSUBSCRIBED', 'SUBSCRIBED'],
      dtype='object')
(82544, 35)
2023-03-26 00:00:00 2024-05-03 00:00:00


In [14]:
# 날짜 기준 필터링
youtube_videos = youtube_videos[youtube_videos['youtube_user_id'].isin(youtube_user_id_inner)].sort_values(['youtube_user_id','end_date']).reset_index(drop=True)

# 날짜 기준 필터링
youtube_videos['end_date'] = pd.to_datetime(youtube_videos['end_date'])
opt_date = (youtube_videos['end_date'] >= first_date) & (youtube_videos['end_date'] <= last_date)
youtube_videos = youtube_videos[opt_date].sort_values(['youtube_user_id', 'end_date']).reset_index(drop=True)

print(youtube_videos.columns)
print(youtube_videos.shape)
print(youtube_videos['end_date'].min(), youtube_videos['end_date'].max())

# youtube_videos.to_csv('C:/py_src/awake/data/youtube_videos.csv',encoding='utf-8-sig',index=False)

Index(['youtube_user_id', 'video', 'end_date', 'views', 'redViews', 'comments',
       'likes', 'dislikes', 'shares', 'estimatedMinutesWatched',
       'estimatedRedMinutesWatched', 'averageViewDuration',
       'averageViewPercentage', 'videosAddedToPlaylists',
       'videosRemovedFromPlaylists', 'estimatedRevenue', 'estimatedAdRevenue',
       'grossRevenue', 'estimatedRedPartnerRevenue', 'playbackBasedCpm', 'cpm',
       'subscribersGained', 'subscribersLost', 'monetizedPlaybacks',
       'adImpressions', 'cardClickRate', 'cardTeaserClickRate',
       'cardImpressions', 'cardTeaserImpressions', 'cardClicks',
       'cardTeaserClicks'],
      dtype='object')
(8569511, 31)
2023-03-26 00:00:00 2024-05-03 00:00:00


#### 계정 분석 데이터셋

In [15]:
youtube_revenue = youtube_revenue.rename(columns={'data_created_at':'date'})
youtube_revenue['date'] = youtube_revenue['date'].astype(str)
youtube_revenue.head()

,youtube_user_id,date,estimated_revenue,estimated_ad_revenue,estimated_red_partner_revenue,gross_revenue,cpm,ad_impressions,monetized_playbacks,playback_based_cpm
0,627cb611aa6f212355e0b617,2023-03-26,9183.544,8153.021,1030.523,14822.121,3687,4020.103,2796,5301.188
1,627cb611aa6f212355e0b617,2023-03-27,7140.192,6217.583,922.609,11276.270,3224,3497.602,2416,4667.330
2,627cb611aa6f212355e0b617,2023-03-28,8044.453,6977.420,1067.034,12613.486,3259,3870.355,2487,5071.768
3,627cb611aa6f212355e0b617,2023-03-29,8027.984,6870.266,1157.717,12489.802,3761,3320.873,2858,4370.120
4,627cb611aa6f212355e0b617,2023-03-30,9723.500,8558.340,1165.160,15444.485,4128,3741.397,3125,4942.235


In [16]:
youtube_channel_demographics = youtube_channel_demographics.rename(columns={'end_date':'date'})
youtube_channel_demographics['date'] = youtube_channel_demographics['date'].astype(str)
youtube_channel_demographics.head()

,youtube_user_id,date,age13-17.female,age13-17.male,age18-24.female,age18-24.male,age25-34.female,age25-34.male,age35-44.female,age35-44.male,age45-54.female,age45-54.male,age55-64.female,age55-64.male,age65-.female,age65-.male
0,627cb611aa6f212355e0b617,2023-03-26,3.6,2.8,23.4,13.7,14.9,13.8,7.7,10.0,4.4,3.7,0.6,0.4,0.6,0.4
1,627cb611aa6f212355e0b617,2023-03-27,4.9,3.7,21.3,15.9,14.5,11.7,10.0,6.5,4.2,7.3,NaN,NaN,NaN,NaN
2,627cb611aa6f212355e0b617,2023-03-28,5.0,3.1,19.8,14.3,17.3,12.1,8.5,8.2,4.1,5.3,0.5,0.7,0.4,0.7
3,627cb611aa6f212355e0b617,2023-03-29,5.2,3.1,18.5,14.8,17.5,12.0,8.8,8.9,4.2,5.4,0.7,1.0,NaN,NaN
4,627cb611aa6f212355e0b617,2023-03-30,3.3,2.4,21.7,13.9,14.8,12.5,9.9,8.8,4.0,6.1,0.7,0.9,0.5,0.6


In [17]:
# 핸드폰 번호 형식으로 변환하는 함수
def format_phone_number(x):
    if pd.isna(x):
        return "010-0000-0000"  # NaN 값을 기본 전화번호 형식으로 대체
    else:
        # 숫자를 정수로 변환한 후 문자열로 변환
        phone_str = str(int(x))
        phone_format = '0' + phone_str

        phone_format = phone_format[:3] + '-' + phone_format[3:7]+ '-' + phone_format[7:]

        
        # 3-3-4 형식으로 포맷팅 (예: 010-1234-5678)
        return phone_format
    
youtube_users['phone_num'] = youtube_users['phone_num'].apply(format_phone_number)
youtube_users.head()

,channel_id,channel_title,phone_num,report_user_id,published_at,viewCount,subscriberCount,videoCount
0,UC-4K3WFY7e_TzaqjXgwxRoA,대신 밍튜브,010-0000-0000,NaN,2013-08-25 04:05:50.000,0,496,0
1,UC-5O4BK-89lZh2r015JBYOg,민경,010-9577-0671,NaN,2018-07-21 16:07:06.000,0,0,0
2,UC-5ra7EcbVUPeNdbaccWJ_A,컴퓨터읽어주는남자 컴읽남,010-4154-8930,NaN,2019-04-11 12:25:55.000,0,0,0
3,UC-8BokR4IeKjUgysUTDutnw,최 이 월,010-0000-0000,6416a206c4d04f017d23e6d7,2023-03-12 14:58:40.264,0,0,0
4,UC-Hk-8YkjlFeEmtdESSbFbA,VoidNeverstop,010-2624-8499,NaN,2014-01-19 06:21:57.000,196505,1160,225


In [18]:
youtube_channel_locations = youtube_channel_locations.rename(columns={'end_date':'date'})
youtube_channel_locations['date'] = youtube_channel_locations['date'].astype(str)
youtube_channel_locations.head()

,youtube_user_id,date,views,estimatedMinutesWatched,averageViewDuration,averageViewPercentage
0,627cb611aa6f212355e0b617,2023-03-26,3941,18721,174.312500,26.291250
1,627cb611aa6f212355e0b617,2023-03-27,3118,15679,185.275862,27.480000
2,627cb611aa6f212355e0b617,2023-03-28,3549,17190,176.277778,25.808333
3,627cb611aa6f212355e0b617,2023-03-29,4037,18420,181.969697,26.713030
4,627cb611aa6f212355e0b617,2023-03-30,4228,18914,163.558824,24.471471


In [19]:
youtube_daily_channel_basics = youtube_daily_channel_basics.rename(columns={'day':'date'})
youtube_daily_channel_basics['date'] = youtube_daily_channel_basics['date'].astype(str)
youtube_daily_channel_basics.head()

,youtube_user_id,date,annotationClickThroughRate,annotationCloseRate,averageViewDuration,comments,dislikes,estimatedMinutesWatched,likes,shares,...,views,redViews,estimatedRevenue,estimatedAdRevenue,estimatedRedPartnerRevenue,grossRevenue,cpm,monetizedPlaybacks,adImpressions,playbackBasedCpm
0,627cb611aa6f212355e0b617,2023-03-26,0.0,0.0,279,4,0,19695,11,5,...,4235,794,7.101,6.304,0.797,11.461,3.108,2796.0,3687.0,4.099
1,627cb611aa6f212355e0b617,2023-03-27,0.0,0.0,292,1,0,16517,16,4,...,3390,686,5.489,4.780,0.709,8.669,2.689,2416.0,3224.0,3.588
2,627cb611aa6f212355e0b617,2023-03-28,0.0,0.0,284,0,2,17988,47,2,...,3795,753,6.194,5.373,0.822,9.712,2.980,2487.0,3259.0,3.905
3,627cb611aa6f212355e0b617,2023-03-29,0.0,0.0,267,4,1,19248,32,4,...,4322,894,6.157,5.269,0.888,9.578,2.547,2858.0,3761.0,3.351
4,627cb611aa6f212355e0b617,2023-03-30,0.0,0.0,262,2,2,20031,44,5,...,4573,860,7.488,6.591,0.897,11.894,2.881,3125.0,4128.0,3.806


In [20]:
youtube_subscriber = youtube_subscriber.rename(columns={'data_created_at':'date'})
youtube_subscriber['date'] = youtube_subscriber['date'].astype(str)
youtube_subscriber.head()

,youtube_user_id,date,subscribers_gained,subscribers_lost,subscribers_count
0,627cb611aa6f212355e0b617,2023-03-26,39,43,122149
1,627cb611aa6f212355e0b617,2023-03-27,16,32,122133
2,627cb611aa6f212355e0b617,2023-03-28,39,29,122143
3,627cb611aa6f212355e0b617,2023-03-29,46,24,122165
4,627cb611aa6f212355e0b617,2023-03-30,44,25,122184


In [21]:
youtube_datas = youtube_datas.rename(columns={'data_created_at':'date'})
youtube_datas['date'] = youtube_datas['date'].astype(str)
youtube_datas.head()

,youtube_user_id,date,published_at,channel_id,channel_title,yt_search_keyword,subscriber_count,video_count,view_count,comment_count,...,YT_PLAYLIST_PAGE,ANNOTATION,NOTIFICATION,PRODUCT_PAGE,SOUND_PAGE,HASHTAGS,SHORTS,ADVERTISING,UNSUBSCRIBED,SUBSCRIBED
0,627cb611aa6f212355e0b617,2023-03-26,NaN,UCxuEudcvmg4zMQhW7isWi-w,NaN,"{'성팩': 40, '빙수빙': 28, '원칩챌린지 먹방': 14, '수빙수': 1...",0.0,0.0,4235,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3356.0,879.0
1,627cb611aa6f212355e0b617,2023-03-27,NaN,UCxuEudcvmg4zMQhW7isWi-w,NaN,"{'원칩챌린지 먹방': 19, '빙수빙': 18, '성팩': 18, 'one chi...",0.0,0.0,3390,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2714.0,676.0
2,627cb611aa6f212355e0b617,2023-03-28,NaN,UCxuEudcvmg4zMQhW7isWi-w,NaN,"{'수빙수': 18, '빙수빙': 17, '성팩': 17, '원칩챌린지 먹방': 1...",0.0,0.0,3795,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3060.0,735.0
3,627cb611aa6f212355e0b617,2023-03-29,NaN,UCxuEudcvmg4zMQhW7isWi-w,NaN,"{'성팩': 45, '빙수빙': 23, '수빙수': 21, '원칩': 13, '원칩...",0.0,0.0,4322,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3504.0,818.0
4,627cb611aa6f212355e0b617,2023-03-30,NaN,UCxuEudcvmg4zMQhW7isWi-w,NaN,"{'원칩챌린지 먹방': 29, '빙수빙': 28, '원칩': 22, '성팩': 16...",0.0,0.0,4573,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3829.0,744.0


In [22]:
# 계정 분석 데이터 테이블
# youtube_revenue
# youtube_channel_demographics
# youtube_users
# youtube_channel_locations
# youtube_daily_channel_basics
# youtube_subscriber
# youtube_datas

In [23]:
# # 계정 분석 데이터 테이블 매핑
merge_df_users = pd.merge(youtube_users,youtube_datas,how='outer',on=['channel_id']).reset_index(drop=True)
merge_df_users = pd.merge(merge_df_users,youtube_revenue,how='outer',on=['youtube_user_id','date']).reset_index(drop=True)
merge_df_users = pd.merge(merge_df_users,youtube_channel_demographics,how='outer',on=['youtube_user_id','date']).reset_index(drop=True)
merge_df_users = pd.merge(merge_df_users,youtube_channel_locations,how='outer',on=['youtube_user_id','date']).reset_index(drop=True)
merge_df_users = pd.merge(merge_df_users,youtube_daily_channel_basics,how='outer',on=['youtube_user_id','date']).reset_index(drop=True)
merge_df_users = pd.merge(merge_df_users,youtube_subscriber,how='outer',on=['youtube_user_id','date']).reset_index(drop=True)
# merge_df_users = pd.merge(merge_df_users,youtube_datas,how='outer',on=['youtube_user_id','date']).reset_index(drop=True)
# merge_df_users = pd.merge(merge_df_users,youtube_users,how='outer',on=['channel_id']).reset_index(drop=True)

merge_df_users = merge_df_users.drop(['channel_title_y','published_at_y'],axis=1)
merge_df_users

,channel_id,channel_title_x,phone_num,report_user_id,published_at_x,viewCount,subscriberCount,videoCount,youtube_user_id,date,...,estimatedAdRevenue,estimatedRedPartnerRevenue,grossRevenue,cpm_y,monetizedPlaybacks,adImpressions,playbackBasedCpm,subscribers_gained,subscribers_lost,subscribers_count
0,UCxuEudcvmg4zMQhW7isWi-w,성팩 SPAAK,010-3906-9284,NaN,2013-07-31 07:43:00.000,14328509.0,118000.0,36.0,627cb611aa6f212355e0b617,2023-03-26,...,6.304,0.797,11.461,3.108,2796.0,3687.0,4.099,39.0,43.0,122149.0
1,UCxuEudcvmg4zMQhW7isWi-w,성팩 SPAAK,010-3906-9284,NaN,2013-07-31 07:43:00.000,14328509.0,118000.0,36.0,627cb611aa6f212355e0b617,2023-03-27,...,4.780,0.709,8.669,2.689,2416.0,3224.0,3.588,16.0,32.0,122133.0
2,UCxuEudcvmg4zMQhW7isWi-w,성팩 SPAAK,010-3906-9284,NaN,2013-07-31 07:43:00.000,14328509.0,118000.0,36.0,627cb611aa6f212355e0b617,2023-03-28,...,5.373,0.822,9.712,2.980,2487.0,3259.0,3.905,39.0,29.0,122143.0
3,UCxuEudcvmg4zMQhW7isWi-w,성팩 SPAAK,010-3906-9284,NaN,2013-07-31 07:43:00.000,14328509.0,118000.0,36.0,627cb611aa6f212355e0b617,2023-03-29,...,5.269,0.888,9.578,2.547,2858.0,3761.0,3.351,46.0,24.0,122165.0
4,UCxuEudcvmg4zMQhW7isWi-w,성팩 SPAAK,010-3906-9284,NaN,2013-07-31 07:43:00.000,14328509.0,118000.0,36.0,627cb611aa6f212355e0b617,2023-03-30,...,6.591,0.897,11.894,2.881,3125.0,4128.0,3.806,44.0,25.0,122184.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106369,UCz60pfE3dOBsTGC2FeUuERg,휘쿡 Hwi Cook,010-0000-0000,NaN,2018-11-19 18:01:36.000,82955.0,265.0,66.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106370,UCz75PDkNQT88rrdHYSacSFg,jihye Choi,010-0000-0000,NaN,2018-08-21 09:43:33.000,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106371,UCzNsDl-geB-n6MFFkqKDftA,맛있는부산 쥰맛지도,010-4320-9436,NaN,2011-11-22 07:34:54.000,651042.0,579.0,94.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106372,UCzYpYFxdQ4XWUfWxE53DrPw,서인아,010-0000-0000,63c159ec4238543bcaf01c56,2013-02-15 12:33:32.000,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
# # 테이블별 필요 컬럼
# 'youtube_user_id', 'date', 
# 'channel_id', 'channel_title_x', 'phone_num', 'report_user_id', 'published_at_x', 'viewCount', 'subscriberCount', 'videoCount',   ## youtube_users

# 'yt_search_keyword','UNSUBSCRIBED', 'SUBSCRIBED', ## youtube_datas

# 'estimated_revenue','estimated_ad_revenue', 
# 'estimated_red_partner_revenue',
# 'gross_revenue', 'cpm_x', 'ad_impressions', 
# 'monetized_playbacks','playback_based_cpm', ## youtube_revenue

# 'age13-17.female', 'age13-17.male',
# 'age18-24.female', 'age18-24.male', 'age25-34.female', 'age25-34.male',
# 'age35-44.female', 'age35-44.male', 'age45-54.female', 'age45-54.male',
# 'age55-64.female', 'age55-64.male', 'age65-.female', 'age65-.male', ## youtube_channel_demographics

# 'averageViewPercentage', ## youtube_channel_locations

# 'annotationClickThroughRate',
# 'annotationCloseRate', 'averageViewDuration_y', 'comments', 'dislikes',
# 'estimatedMinutesWatched_y', 'likes', 'shares', 'subscribersGained',
# 'subscribersLost', 'views_y', 'redViews', 'estimatedRevenue',
# 'estimatedAdRevenue', 'estimatedRedPartnerRevenue', 'grossRevenue',
# 'cpm_y', 'monetizedPlaybacks', 'adImpressions', 'playbackBasedCpm', ## youtube_daily_channel_basics


# 'subscribers_gained', 'subscribers_lost', 'subscribers_count' ## youtube_subscriber

In [25]:
# 최종 컬럼 정리
users_col_need = [
    # 계정, 날짜 기준
    'youtube_user_id', 'date', 

    # 계정 정보
    'channel_id', 'channel_title_x', 'phone_num', 'report_user_id', 'published_at_x', 'viewCount', 'subscriberCount', 'videoCount', 'yt_search_keyword',

    # 참여도 관련
    'views_y','redViews','UNSUBSCRIBED', 'SUBSCRIBED', 'comments', 'likes', 'shares', 'dislikes', 'estimatedMinutesWatched_y', 'averageViewDuration_y', 'averageViewPercentage',

    # 수익 관련
    'estimated_revenue', 'estimatedRevenue', 'estimated_ad_revenue', 'estimatedAdRevenue', 'estimated_red_partner_revenue', 'estimatedRedPartnerRevenue', 
    'gross_revenue', 'grossRevenue', 'cpm_x', 'cpm_y', 'playback_based_cpm', 'playbackBasedCpm',

    # 구독자 관련
    'subscribers_count',
    'subscribersGained', 'subscribers_gained',
    'subscribersLost', 'subscribers_lost',

    # 광고 관련
    'ad_impressions', 'adImpressions',
    'monetized_playbacks', 'monetizedPlaybacks',

    # 연령 및 성별 관련
    'age13-17.female', 'age13-17.male',
    'age18-24.female', 'age18-24.male', 'age25-34.female', 'age25-34.male',
    'age35-44.female', 'age35-44.male', 'age45-54.female', 'age45-54.male',
    'age55-64.female', 'age55-64.male', 'age65-.female', 'age65-.male'
]
len(users_col_need)

57

In [34]:
# 최종 계정 분석 데이터셋

# 컬럼명 변경
merge_df_users_fin = merge_df_users[users_col_need]
merge_df_users_fin = merge_df_users_fin.rename(columns={'channel_title_x':'channel_title', 'published_at_x':'published_at', 'views_y':'views', 'cpm_x' : 'cpm',
                                                        'estimatedMinutesWatched_y':'estimatedMinutesWatched', 'averageViewDuration_y':'averageViewDuration'})

# 전처리
# 환율 적용 - 해당기간 평균환율 : 1322.42
exchange_rate_col = ['estimatedRevenue', 'estimatedAdRevenue', 'estimatedRedPartnerRevenue', 'grossRevenue', 'cpm_y', 'playbackBasedCpm']
merge_df_users_fin[exchange_rate_col] = merge_df_users_fin[exchange_rate_col] * 1322.42

# 시청 시간 단위 통일
merge_df_users_fin['averageViewDuration'] = round(merge_df_users_fin['averageViewDuration'] / 60, 2) ## 초 단위 분으로 변경

# null값 평균환율 적용값으로 대체
merge_df_users_fin['estimated_revenue'] = merge_df_users_fin['estimated_revenue'].fillna(merge_df_users_fin['estimatedRevenue'])
merge_df_users_fin['estimated_ad_revenue'] = merge_df_users_fin['estimated_ad_revenue'].fillna(merge_df_users_fin['estimatedAdRevenue'])
merge_df_users_fin['estimated_red_partner_revenue'] = merge_df_users_fin['estimated_red_partner_revenue'].fillna(merge_df_users_fin['estimatedRedPartnerRevenue'])
merge_df_users_fin['gross_revenue'] = merge_df_users_fin['gross_revenue'].fillna(merge_df_users_fin['grossRevenue'])
merge_df_users_fin['cpm'] = merge_df_users_fin['cpm'].fillna(merge_df_users_fin['cpm_y'])
merge_df_users_fin['playback_based_cpm'] = merge_df_users_fin['playback_based_cpm'].fillna(merge_df_users_fin['playbackBasedCpm'])

# null값 정확한 테이블값으로 대체
merge_df_users_fin['ad_impressions'] = merge_df_users_fin['ad_impressions'].fillna(merge_df_users_fin['adImpressions'])
merge_df_users_fin['subscribers_gained'] = merge_df_users_fin['subscribers_gained'].fillna(merge_df_users_fin['subscribersGained'])
merge_df_users_fin['subscribers_lost'] = merge_df_users_fin['subscribers_lost'].fillna(merge_df_users_fin['subscribersLost'])
merge_df_users_fin['monetized_playbacks'] = merge_df_users_fin['monetized_playbacks'].fillna(merge_df_users_fin['monetizedPlaybacks'])

# 대체된 값 제거
merge_df_users_fin = merge_df_users_fin.drop(['estimatedRevenue', 'estimatedAdRevenue', 'estimatedRedPartnerRevenue', 'grossRevenue', 'cpm_y', 'playbackBasedCpm',
                                              'adImpressions','subscribersGained','subscribersLost','monetizedPlaybacks'],axis=1)

# 나머지 null값 측정안된값으로 0으로 대체
merge_df_users_fin[merge_df_users_fin.columns[11:]] = merge_df_users_fin[merge_df_users_fin.columns[11:]].fillna(0)

In [36]:
# merge_df_users_fin.to_csv('C:/py_src/awake/data/merge_df_users_fin.csv', encoding='utf-8-sig',index=False)

#### 콘텐츠 분석 데이터셋

In [ ]:
# 불필요컬럼 제거 - 모두 0인 컬럼
youtube_videos = youtube_videos.drop(['annotationClickThroughRate', 'annotationCloseRate',
                                      'annotationImpressions', 'annotationClickableImpressions',
                                      'annotationClosableImpressions', 'annotationClicks', 'annotationCloses'],axis=1)

In [ ]:
# 최종 컬럼 정리
video_col_need = [
    # 콘텐츠, 날짜 기준
    'youtube_user_id', 'video', 'end_date',

    # 참여도 관련
    'views', 'redViews', 'comments', 'likes', 'dislikes', 'shares', 'estimatedMinutesWatched', 'estimatedRedMinutesWatched', 'averageViewDuration', 'averageViewPercentage', 
    'videosAddedToPlaylists','videosRemovedFromPlaylists',

    # 수익 관련
    'estimatedRevenue', 'estimatedAdRevenue', 'grossRevenue','estimatedRedPartnerRevenue', 'playbackBasedCpm', 'cpm',

    # 구독자 관련
    'subscribersGained', 'subscribersLost',

    # 광고 관련
    'monetizedPlaybacks', 'adImpressions',

    # 카드 관련
    'cardClickRate', 'cardTeaserClickRate','cardImpressions', 'cardTeaserImpressions', 'cardClicks', 'cardTeaserClicks'
]
len(video_col_need)

31

In [ ]:
# 최종 데이터셋 정리
youtube_videos = youtube_videos[video_col_need].sort_values(['video', 'end_date']).reset_index(drop=True)

In [ ]:
youtube_videos.head()

,youtube_user_id,video,end_date,views,redViews,comments,likes,dislikes,shares,estimatedMinutesWatched,...,subscribersGained,subscribersLost,monetizedPlaybacks,adImpressions,cardClickRate,cardTeaserClickRate,cardImpressions,cardTeaserImpressions,cardClicks,cardTeaserClicks
0,64467ea09634a10e3709e1ea,--0HSDH6J7o,2023-04-23,3,1,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0,0,0,0
1,64467ea09634a10e3709e1ea,--0HSDH6J7o,2023-04-30,4,0,0,0,0,0,1,...,0,0,0.0,0.0,0.0,0.0,0,0,0,0
2,64467ea09634a10e3709e1ea,--0HSDH6J7o,2023-05-01,2,1,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0,0,0,0
3,64467ea09634a10e3709e1ea,--0HSDH6J7o,2023-05-02,3,0,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0,0,0,0
4,64467ea09634a10e3709e1ea,--0HSDH6J7o,2023-05-03,1,0,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0,0,0,0


In [ ]:
# youtube_videos.to_csv('C:/py_src/awake/data/youtube_videos.csv', encoding='utf-8-sig',index=False)